<a href="https://colab.research.google.com/github/ucfilho/AI/blob/main/AI_2023/Stochastic/class_04_hen_v2_april_04_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
Target = {'H1':[400,60],'H2':[210,40],'C1':[20,160],'C2':[100,300]} # temperatures initial and target
Util = {'cold':[20,30],'hot':[400,400]}

# Temperatures (hot streams first, cold after and starting with H/C for hot and cold) otherwise need to adapt the code
Prop = {'H1':[0.3,1.7],'H2':[0.5,1.7],'C1':[0.4,1.7],'C2':[0.6,1.7],'cold':[4.186e-3,1.7],'hot':[431.379,3.41]} # CP e H
w = [0.5,0.3,0.8,0.5,0.8,0.8,0.55,0.15,0.15,0.12,0.13,0.08]
#w = [0.5,0.3,0.8,0.5,0.8,0.8,0.99,0.99,0.99,0.99,0.99,0.08]
# w input of swarm intelligence size = n_exchages*num_hot
n_trocas = 3


In [3]:
def Cost_find(w,Target = Target, Prop = Prop, Util= Util, n_trocas = n_trocas, ans = False ):
  Gasta_cold = [] # [Area,flow,kwy] for each stream
  Gasta_hot = []  # [Area,flow,kwy] for each stream
  U_q =[]
  names = list(Target.keys())
  n_hot, n_cold, stream= 0, 0, ['No']
  # constroi temperaturas
  Temp ={}
  for word in names:
    Temp[word]=[Target[word][0]]

  for word in names:
    if 'H' in word:
      n_hot += 1
    elif 'C' in word:
      n_cold += 1
      stream.append(word)

  N = len(stream)
  r = 1/N
  trocas =[]
  Areas = []
  n = n_trocas*n_hot

  x = w[0:n]
  tx = w[n:(2*n)]
  cont = 0
  for xi in x:
    total = 0
    cont += 1
    for k in range(N):
      total += r
      if xi <= total:
        trocas.append(stream[k])
        break 

  n1, n2, matrix_trocas, xt_hot = 0, 0, [],[]
  num = int(len(trocas)/n_hot)
  for idx in range(n_hot):
    tr, dr =  [], []
    n1 += n2
    n2 = num*(1+idx)
    tr.append(trocas[n1:n2])
    dr.append(tx[n1:n2])
    matrix_trocas.append(tr[0])
    xt_hot.append(dr[0])
  idx = 0
  # update hot temperature
  for j in range(n_trocas):
    for k in range(n_hot):
      #print('======= idx k names[k]',idx,k,names[k], Temp)
      x = Temp[names[k]][idx]
      # print('x',x)
      cold = matrix_trocas[k][j]
      hot = names[k]
      if cold == 'No':
        Temp[names[k]].append(x)
      else:
        # x, y = hot and cold temperatures
        dx = Target[hot][0]-Target[hot][1]
        x0 = x* 1.0
        y0 = Temp[cold][-1]
        x = x - dx * xt_hot[k][j]
        CP_c = Prop[cold][0]
        CP_h = Prop[hot][0]
        #print('cold & hot',cold,hot)
        #print('CP===',CP_c,CP_h)
        Q = CP_c* dx * xt_hot[k][j]
        y = CP_h*dx * xt_hot[k][j]/CP_c + Temp[cold][-1]
        d1 = x0- y
        d2 = x - y0
        U = 1.0 / ( 1.0/Prop[cold][1]+ 1.0/Prop[hot][1])
        #print('U=',U,cold,hot)
        if d2 != 0 and d1*d2 > 0:
          check = 'ok'
        else:
          check = 'no'
        if x0 > y and x > y0 and check == 'ok': # so troca calor se a temperatura fria final for menor q a temperatura quente final
          Temp[cold].append(y)
          Temp[names[k]].append(x)
          LMDT = (d1 - d2) / np.log(d1/d2)
          A = Q /(U*LMDT)* 1000 # convert MW to kW
          Areas.append(A)
          U_q.append(U)
        else:
          y = Temp[cold][-1]
          x = Temp[names[k]][-1]
          Temp[cold].append(y)
          Temp[names[k]].append(x)       
    idx += 1

  cp_hot = Prop['hot'][0] # cp
  dt = Util['cold'][1]-Util['cold'][0]
  cp_cold = Prop['cold'][0]*dt 

  for word in names:
    x0 = Temp[word][-1]
    x = Target[word][1]
    cp = Prop[word][0]
    Q = cp*abs(x-x0)
    
    if x0 < x : # cooling activate
      flow = Q/cp_cold
      U = 1.0 / ( 1.0/Prop[word][1]+ 1.0/Prop['cold'][1])
      y0 = Util['cold'][0]
      y = Util['cold'][1]
      d1 = x0- y
      d2 = x - y0   
      if d2 != 0 and d1*d2 > 0:
        check = 'ok'
      else:
        check = 'no'
      if x0 > y and x > y0 and check == 'ok': # so troca calor se a temperatura fria final for menor q a temperatura quente final
        LMDT = (d1 - d2) / np.log(d1/d2)
        A = Q /(U*LMDT)* 1000 # convert MW to kW
        MWh = Q*8000
        Gasta_cold.append([A,flow,MWh])   
    else:
      flow = Q / cp_hot 
      U = 1.0 / ( 1.0/Prop[word][1]+ 1.0/Prop['cold'][1])
      y0 = Util['hot'][0]
      y = Util['hot'][1]
      d1 = y-x0 
      d2 = y0  -x   
      if d2 != 0 and d1*d2 > 0:
        check = 'ok'
      else:
        check = 'no'
      if x0 < y and x < y0 and check == 'ok': # so troca calor se a temperatura fria final for menor q a temperatura quente final
        LMDT = (d1 - d2) / np.log(d1/d2)
        A = Q /(U*LMDT)* 1000 # convert MW to kW
        kWy= Q*8000
        Gasta_hot.append([A,flow,kWy])   

  Cost = 0
  a, n, h, c = 4333,0.6, 150.53, 53.064 # L.V. Pavao et al. / Energy 142 (2018) 356e372
  for A in Areas:
    Cost += a*A**n
  for x in Gasta_hot:
    Cost += a*x[0]**n # x[0]:area
    Cost += h*x[2] # x[2]: steam 

  for x in Gasta_cold:
    Cost += a*x[0]**n # x[0]:area
    Cost += c*x[2] # x[2]: cold water kwy

  if ans == False:
    return Cost
  else:
    return Cost, matrix_trocas, Temp

In [4]:
Cost = Cost_find(w)
print('Cost',Cost)

print(Cost_find(w,ans=True))

Cost 156009552.4550301
(156009552.4550301, [['C1', 'No', 'C2'], ['C1', 'C2', 'C2']], {'H1': [400, 212.99999999999997, 212.99999999999997, 161.99999999999997], 'H2': [210, 189.6, 167.5, 153.9], 'C1': [20, 160.25, 185.75], 'C2': [100, 118.41666666666667, 143.91666666666669, 155.25000000000003]})


In [5]:
# https://www.tlv.com/global/BR/calculator/steam-table-temperature.html